In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dropout,Conv2DTranspose,Dense,BatchNormalization,Reshape,Activation,Conv2D,MaxPooling2D,Flatten,Input
from keras.models import Sequential,Model
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
 (x_train, y_train), (x_test, y_test) =  mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [50]:
def load_data():
    (x_train, y_train), (x_test, y_test) =  mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    # convert shape of x_train from (60000, 32 ,32) to (60000, 784) 
    # 784 columns per row
    x_train = x_train.reshape(60000,28,28,1)
    return (x_train, y_train, x_test, y_test)
(X_train, y_train,X_test, y_test)=load_data()
print(X_train.shape)

(60000, 28, 28, 1)


In [57]:
def create_generator():
  epsilon = 0.00001 
  model = Sequential()
  model.add(Dense(7*7*256, input_shape=(100,)))
  model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Reshape((7,7,256)))
  model.add(Conv2DTranspose(128,(5,5),strides=[1,1],padding='same'))
  model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(64,(5,5),strides=[2,2],padding='same'))
  model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(1,(5,5),strides=[2,2],activation='tanh',padding='same'))
  model.add(BatchNormalization(momentum=0.9, epsilon=epsilon))
  model.add(LeakyReLU(alpha=0.2))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
  model.summary()
  return model

In [58]:
g = create_generator()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_48 (Batc (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_70 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_20 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_25 (Conv2DT (None, 7, 7, 128)         819328    
_________________________________________________________________
batch_normalization_49 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_71 (LeakyReLU)   (None, 7, 7, 128)       

In [63]:
def create_discriminator():
  model = Sequential()
  model.add(Conv2D(64, (3,3), padding='same', input_shape=(28,28,1)))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.2))

  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(128))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(128))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
  model.summary()
  return model

In [64]:
d = create_discriminator()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
leaky_re_lu_90 (LeakyReLU)   (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_64 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
leaky_re_lu_91 (LeakyReLU)   (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_65 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 9, 9, 64)        

In [65]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
gan = create_gan(d,g)
gan.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_25 (Sequential)   (None, 28, 28, 1)         2343685   
_________________________________________________________________
sequential_29 (Sequential)   (None, 1)                 424769    
Total params: 2,768,454
Trainable params: 2,318,211
Non-trainable params: 450,243
_________________________________________________________________


In [ ]:
def training(epochs=1, batch_size=128):
    
    #Loading the data
    (X_train, y_train, X_test, y_test) = load_data()
    batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 100])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            print("generated images",generated_images.shape)
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            print("image batch",image_batch.shape)
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
           
            plot_generated_images(e, generator)
training(400,128)

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_68 (Batc (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_96 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_22 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_31 (Conv2DT (None, 7, 7, 128)         819328    
_________________________________________________________________
batch_normalization_69 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_97 (LeakyReLU)   (None, 7, 7, 128)       

  0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1
generated images (128, 28, 28, 1)
image batch (128, 28, 28, 1)


  1%|          | 1/128 [00:10<21:24, 10.12s/it]

generated images (128, 28, 28, 1)
image batch (128, 28, 28, 1)


...

generated images (128, 28, 28, 1)
image batch (128, 28, 28, 1)


  5%|▍         | 6/128 [00:30<10:17,  5.07s/it]

generated images (128, 28, 28, 1)
image batch (128, 28, 28, 1)


  5%|▌         | 7/128 [00:34<09:34,  4.75s/it]

generated images (128, 28, 28, 1)
image batch (128, 28, 28, 1)
